In [ ]:
!nvidia-smi 

Fri Aug 26 13:34:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers -q

     |████████████████████████████████| 4.7 MB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
     |████████████████████████████████| 120 kB 65.0 MB/s 


In [ ]:
!pip install datasets -q

     |████████████████████████████████| 365 kB 7.7 MB/s 
     |████████████████████████████████| 115 kB 62.7 MB/s 
     |████████████████████████████████| 212 kB 60.1 MB/s 
     |████████████████████████████████| 127 kB 70.5 MB/s 


##Creating Data

In [ ]:
!wget https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/train.json

--2022-08-26 13:35:13--  https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4562448 (4.4M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]   4.35M  --.-KB/s    in 0.06s   

2022-08-26 13:35:14 (76.1 MB/s) - ‘train.json’ saved [4562448/4562448]



In [ ]:
!wget https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/test.json

--2022-08-26 13:35:14--  https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4562448 (4.4M) [text/plain]
Saving to: ‘test.json’

test.json           100%[===================>]   4.35M  --.-KB/s    in 0.05s   

2022-08-26 13:35:14 (91.8 MB/s) - ‘test.json’ saved [4562448/4562448]



In [ ]:
!wget https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/validation.json

--2022-08-26 13:35:14--  https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/validation.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4562448 (4.4M) [text/plain]
Saving to: ‘validation.json’

validation.json     100%[===================>]   4.35M  --.-KB/s    in 0.04s   

2022-08-26 13:35:15 (104 MB/s) - ‘validation.json’ saved [4562448/4562448]



In [ ]:
import json
from datasets import load_dataset



def convert_to_jsonl(input_filename, output_filename):

  with open(input_filename) as f:
      dataset = json.load(f)

  with open(output_filename, "w") as f:
      for article in dataset["data"]:
          title = article["title"]
          for paragraph in article["paragraphs"]:
              context = paragraph["context"]
              answers = {}
              for qa in paragraph["qas"]:
                  question = qa["question"]
                  idx = qa["id"]
                  answers["text"] = [a["text"] for a in qa["answers"]]
                  answers["answer_start"] = [a["answer_start"] for a in qa["answers"]]
                  f.write(
                      json.dumps(
                          {
                              "id": idx,
                              "title": title,
                              "context": context,
                              "question": question,
                              "answers": answers,
                          }
                      )
                  )
                  f.write("\n")



In [ ]:
convert_to_jsonl("train.json","train.jsonl")

In [ ]:
convert_to_jsonl("test.json","test.jsonl")

In [ ]:
convert_to_jsonl("validation.json","validaiton.jsonl")

In [ ]:
from dataclasses import field
from datasets import load_dataset

data_files = {
    "train":"train.jsonl",
    "test":"test.jsonl",
    "validation":"validaiton.jsonl"
}

raw_datasets = load_dataset("json",data_files = data_files)
raw_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-188ed520f496729b/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2504
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2504
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2504
    })
})

In [ ]:
raw_datasets["train"][2]

{'id': 'bengali--981248442377505718-0-2649',
 'title': 'শেখ মুজিবুর রহমান',
 'context': 'শেখ মুজিবুর রহমান (১৭ মার্চ ১৯২০ - ১৫ আগস্ট ১৯৭৫) বাংলাদেশের প্রথম রাষ্ট্রপতি ও ভারতীয় উপমহাদেশের একজন অন্যতম প্রভাবশালী রাজনৈতিক ব্যক্তিত্ব যিনি বাঙালীর অধিকার রক্ষায় ব্রিটিশ ভারত থেকে ভারত বিভাজন আন্দোলন এবং পরবর্তীতে  পূর্ব পাকিস্তান থেকে বাংলাদেশ প্রতিষ্ঠার সংগ্রামে নেতৃত্ব প্রদান করেন। প্রাচীন বাঙ্গালি সভ্যতার আধুনিক স্থপতি হিসাবে শেখ মুজিবুর রহমানকে বাংলাদেশের জাতির জনক বা জাতির পিতা বলা হয়ে থাকে। তিনি মাওলানা আব্দুল হামিদ খান ভাসানী প্রতিষ্ঠিত আওয়ামী লীগের সভাপতি, বাংলাদেশের প্রথম রাষ্ট্রপতি এবং পরবর্তীতে এদেশের প্রধানমন্ত্রীর দায়িত্ব পালন করেন। জনসাধারণের কাছে তিনি শেখ মুজিব এবং শেখ সাহেব হিসাবে বেশি পরিচিত এবং তার উপাধি বঙ্গবন্ধু। তার কন্যা শেখ হাসিনা বাংলাদেশ আওয়ামী লীগের বর্তমান সভানেত্রী এবং বাংলাদেশের বর্তমান প্রধানমন্ত্রী।',
 'question': 'শেখ মুজিবুর রহমান কবে জন্মগ্রহণ করেন ?',
 'answers': {'text': ['১৭ মার্চ ১৯২০'], 'answer_start': [19]}}

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,id,title,context,question,answers
0,bengali--6690835373278814985-4-3185,বাংলাদেশের মাছের তালিকা,\nবাংলাদেশে স্বাদু পানির মৎস্য প্রজাতিসমূহের বিজ্ঞানভিত্তিক গবেষণা শুরু হয় ১৮২২ সালে। ২০০৫ সালে এ. কে. আতাউর রহমান বাংলাদেশের স্বাদুপানির মাছকে ৫৫টি পরিবারের অধীনে ১৫৪ গণের ২৬৫টি প্রজাতিকে তালিকাভুক্ত করেছিলেন যার ভেতরে কয়েক প্রজাতির সামুদ্রিক মাছও ছিলো। সর্বশেষ বাংলাদেশ এশিয়াটিক সোসাইটি কর্তৃক প্রকাশিত বাংলাদেশ উদ্ভিদ ও প্রাণী জ্ঞানকোষ-এর ২৩ নং খণ্ডে ১৭টি বর্গের অধীন ৬১টি পরিবারের ২৫১টি প্রজাতিকে স্বাদুপানির মাছ হিসেবে উল্লেখ করা হয়েছে।[3],পৃথিবীতে মোট কয়প্রকার বাসা পরজীবী মাছ আছে ?,"{'text': [], 'answer_start': []}"
1,bengali--1474181390389027325-8-4673,বালিশ,"\n\nবালিশের ভেতর ভাগ ফোম, কৃত্রিম প্লাস্টিকের তন্তু, পালক বা নরম বস্তু এবং ভিস্কোইলাস্টিক ফোম ও রাবারের নির্যাস থেকে তৈরী ফিলার দিয়ে গঠিত। [5] ঐতিহাসিকভাবে খড় দ্বারা বালিশ পুর্ন হতো, কিন্তু এটা অসুবিধাজনক এবং বর্তমানে কম ব্যবহৃত হয়। পালক ও নরম বস্তু সবচেয়ে ব্যায়বহুল এবং সবচেয়ে আরামদায়ক; এগুলোর সুবিধা হল এগুলো নরম এবং ফোম বা তন্তু বালিশের চেয়ে ব্যবহারকারিদের সুবিধা মত আকৃতি পরিবর্তনের সক্ষম। পালক ভর্তি বালিশের অন্যতম অসুবিধা হল ব্যবহারকারিদের এলার্জিক প্রতিক্রিয়া। বর্তমানে হাইপোএলার্জিক প্রজাতির পালকের বালিশ আছে যা পালকের প্রতি সংবেদনশীল মানুষকে পালক বা নরম বালিশের আরাম উপভোগের সুযোগ করে দেয়। এশিয়ায় বাজরা ও প্লাস্টিকের নকল একটি সাধারন ফিলার।এরকম বালিশ সাধারন বালিশের চেয়ে ছোট হয়। ভারতে তুলা একটি সাধারন ফিলার।",তোশক কি তুলো দিয়ে তৈরী হয় ?,"{'text': [], 'answer_start': []}"
2,bengali-2660961368441478017-15-4990,বিভিন্ন ধর্ম,"জৈনধর্ম (সংস্কৃত: जैन धर्म) প্রাচীন ভারতে প্রবর্তিত একটি ধর্মমত। বর্তমানে বিশ্বের নানা দেশে এই ধর্মমতাবলম্বীদের দেখা যায়। জৈনধর্মের মূল বক্তব্য হল সকল জীবের প্রতি শান্তি ও অহিংসার পথ গ্রহণ। জৈন দর্শন ও ধর্মানুশীলনের মূল কথা হল দৈব চৈতন্যের আধ্যাত্মিক সোপানে স্বচেষ্টায় আত্মার উন্নতি। যে ব্যক্তি বা আত্মা অন্তরের শত্রুকে জয় করে সর্বোচ্চ অবস্থা প্রাপ্ত হন তাঁকে জিন (জিতেন্দ্রিয়) আখ্যা দেওয়া হয়। প্রাচীন ধর্মগ্রন্থগুলিতে জৈনধর্মকে শ্রমণ ধর্ম বা নির্গ্রন্থদের ধর্মও বলা হয়েছে। কথিত আছে, তীর্থঙ্কর নামে চব্বিশ জন মহাজ্ঞানী কৃচ্ছ্বসাধকের একটি ধারা পর্যায়ক্রমে জৈনধর্মকে পুনরুদ্ধার করেছিলেন।[34] এঁদের মধ্যে ত্রাবিংশ তীর্থঙ্কর ছিলেন পার্শ্বনাথ (খ্রিষ্টপূর্ব নবম শতাব্দী) ও সর্বশেষ তীর্থঙ্কর ছিলেন মহাবীর (খ্রিষ্টপূর্ব ষষ্ঠ শতাব্দী)।[35][36][37][38][39] আধুনিক বিশ্বে জৈনধর্মাবলম্বীদের সংখ্যা তুলনামূলকভাবে অনেক কম হলেও এই ধর্ম বেশ প্রভাবশালী। ভারতে জৈন ধর্মবলম্বীদের সংখ্যা প্রায় ১০,২০০,০০০।[40] এছাড়া উত্তর আমেরিকা, পশ্চিম ইউরোপ, দূরপ্রাচ্য, অস্ট্রেলিয়া ও বিশ্বের অন্যত্রও অভিবাসী জৈনদের দেখা মেলে।[41]",জৈন ধর্মে মোট কতজন তীর্থঙ্করের কথা বলা হয়েছে ?,"{'text': ['চব্বিশ জন'], 'answer_start': [507]}"
3,bengali-6778942552911693577-2-4078,ভারত মহাসাগর,"ভারত মহাসাগরের ঘনত্ব ২৯২,১৩১,০০০ ঘন কিলোমিটার (৭০,০৮৬,০০০ ঘন মাইল)।[7] মহাসাগরের মহাদেশীয় প্রান্তসীমায় অনেক ছোটো ছোটো দ্বীপ অবস্থিত। ভারত মহাসাগরে অবস্থিত দ্বীপরাষ্ট্রগুলি হল মাদাগাস্কার (বিশ্বের চতুর্থ বৃহত্তম দ্বীপ), রিইউনিয়ন দ্বীপ, কোমোরোস, সেশেল, মালদ্বীপ, মরিশাস ও শ্রীলঙ্কা। ইন্দোনেশিয়া দ্বীপপুঞ্জ এই মহাদেশের পূর্ব সীমায় অবস্থিত।",ইসরায়েলের জাফা সড়কের দৈর্ঘ্য কত ?,"{'text': [], 'answer_start': []}"
4,bengali--497186371677567635-0-3945,ভাগবত পুরাণ,"ভাগবত পুরাণ (দেবনাগরী: भागवतपुराण; অন্যান্য নাম শ্রীমদ্ভাগবত মহাপুরাণ, শ্রীমদ্ভাগবতম্‌ বা ভাগবত; অর্থাৎ, পরমেশ্বরের পবিত্র কাহিনি) হল একটি হিন্দু মহাপুরাণ। এটি একটি ভক্তিবাদী ধর্মগ্রন্থ। বিষ্ণুর পূর্ণ অবতার তথা ""স্বয়ং ভগবান"" কৃষ্ণের প্রতি গভীর ব্যক্তিগত ভক্তিই এই পুরাণের প্রধান আলোচ্য বিষয়।[1] হিন্দু পৌরাণিক সাহিত্যের অনেক কাহিনি তথা বিষ্ণুর চব্বিশ জন অবতারের কাহিনি ভাগবত পুরাণে লিপিবদ্ধ রয়েছে। ভাগবত পুরাণ</i>ই প্রথম পুরাণ যেটি কোনো ইউরোপীয় ভাষায় অনূদিত হয়। ১৮৪০ থেকে ১৮৫৭ সালের মধ্যে ভাগবত পুরাণে</i>র তিনটি ফরাসি অনুবাদ প্রকাশিত হয়।[2] পদ্মপুরাণের শ্রেণিবিন্যাস অনুসারে ভাগবত পুরাণ একটি সাত্ত্বিক পুরাণ (অর্থাৎ, এই পুরাণ কল্যাণ ও পবিত্রতার সঙ্গে যুক্ত)।[3] প্রচলিত হিন্দু বিশ্বাস অনুসারে, ব্যাস এই পুরাণের রচয়িতা।",ভাগবত পুরাণ কোন ভাষায় রচিত হয় ?,"{'text': ['দেবনাগরী'], 'answer_start': [13]}"
5,bengali-6838863342707702200-6-3928,পশ্চিমবঙ্গের জে

In [ ]:
model_checkpoint = "deepset/xlm-roberta-large-squad2"
batch_size = 4

In [ ]:
!pip install sentencepiece -q

     |████████████████████████████████| 1.3 MB 6.5 MB/s 


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# check if the tokenizer is fast tokenizer

import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("আপনার নাম কি?", "আমার নাম ফয়সাল")

{'input_ids': [0, 21742, 37564, 14403, 32, 2, 2, 29388, 37564, 22741, 2801, 16984, 2763, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context 

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    
    #removing left whitespaces
    examples["question"] = [q.lstrip() for q in examples["question"]]


    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

   
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        
        sequence_ids = tokenized_examples.sequence_ids(i)

        
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

           
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = raw_datasets.map(
    prepare_train_features, batched=True, remove_columns=raw_datasets["train"].column_names
)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets["train"].column_names

['input_ids', 'attention_mask', 'start_positions', 'end_positions']

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
model.config

XLMRobertaConfig {
  "_name_or_path": "deepset/xlm-roberta-large-squad2",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [ ]:
!pip install wandb -q

     |████████████████████████████████| 1.8 MB 6.8 MB/s 
     |████████████████████████████████| 157 kB 12.3 MB/s 
     |████████████████████████████████| 181 kB 10.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 157 kB 13.1 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 78.5 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 156 kB 72.1 MB/s 


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb 
from google.colab import drive
import os

model_name = model_checkpoint.split("/")[-1]


drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_name

'xlm-roberta-large-squad2'

In [ ]:
!ls

gdrive	     test.json	 train.json   validaiton.jsonl
sample_data  test.jsonl  train.jsonl  validation.json


In [ ]:
wandb_dir = f"/content/gdrive/MyDrive/DL_models/language_model_bn/wandb_files/"

wandb.init(dir=os.getenv("WANDB_DIR",wandb_dir),project = "language-model-bn",id ="xlm-roberta-deepset",resume="allow")

wandb: Currently logged in as: afschowdhury. Use `wandb login --relogin` to force relogin


In [ ]:
# Training Arguments
args = TrainingArguments(
    seed = 42,
    output_dir = f"/content/gdrive/MyDrive/DL_models/language_model_bn/{model_name}-test-1",
    evaluation_strategy = "steps", #default step 500
    learning_rate=2e-5,
    lr_scheduler_type = "linear",
    warmup_ratio = 0.2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy = "steps",
    save_steps = 1000,
    save_total_limit=2,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    report_to = "wandb",
    run_name ="language-model-bn(qas)_deepset_xlm"
    
)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2828
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2828
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2828
    })
})

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model=model,
    args = args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using cuda_amp half precision backend


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    2131 MB |    2131 MB |    2131 MB |       0 B  |\n|       from large pool |    2130 MB |    2130 MB |    2130 MB |       0 B  |\n|       from small pool |       1 MB |       1 MB |       1 MB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    2131 MB |    2131 MB |    2131 MB |       0 B  |\n|       from large pool |    2130 MB |    2130 MB |

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2828
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2121
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
500,0.960900,0.548447
1000,0.770800,0.450876
1500,0.547400,0.269723
2000,0.302300,0.114284


***** Running Evaluation *****
  Num examples = 2828
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2828
  Batch size = 4
Saving model checkpoint to /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/checkpoint-1000
Configuration saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/checkpoint-1000/config.json
Model weights saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2828
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2828
  Batch size = 4
Saving mode

TrainOutput(global_step=2121, training_loss=0.621309877509837, metrics={'train_runtime': 1601.8068, 'train_samples_per_second': 5.297, 'train_steps_per_second': 1.324, 'total_flos': 5909362590910464.0, 'train_loss': 0.621309877509837, 'epoch': 3.0})

In [ ]:
trainer.save_model(f"/content/gdrive/MyDrive/DL_models/language_model_bn/{model_name}-test-1/afs_xlm_roberta")

Saving model checkpoint to /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/afs_xlm_roberta
Configuration saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/afs_xlm_roberta/config.json
Model weights saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/afs_xlm_roberta/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/afs_xlm_roberta/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/afs_xlm_roberta/special_tokens_map.json


### Inferencing

In [ ]:
model_location = f"/content/gdrive/MyDrive/DL_models/language_model_bn/{model_name}-test-1/afs_xlm_roberta"

In [ ]:
model_location

'/content/gdrive/MyDrive/DL_models/language_model_bn/xlm-roberta-large-squad2-test-1/afs_xlm_roberta'

In [ ]:
from transformers import  pipeline




nlp = pipeline('question-answering', model=model_location, tokenizer=model_location)
QA_input = {
    'question': 'আপনার নাম কি?',
    'context': 'আমার নাম দীপ সাহা। আমি ইসলামী বিশ্ববিদ্যালয়ে পড়ি'
}
res = nlp(QA_input)
res

In [ ]:

QA_input = {
    'question': 'মেক্সিকো সিটি প্রথম কোন বছর প্যান আমেরিকা গেমস আয়োজন করেছিল?',
    'context': 'মেক্সিকো সিটি একমাত্র লাতিন আমেরিকান শহর হিসেবে অলিম্পিক গেমস আয়োজন করেছে। ১৯৬৮ সালে গ্রীষ্মকালীন অলিম্পিক আয়োজন করে তারা। (এটাও পরিবর্তন হবে রিওকে ধন্যবাদ, ২০১৬ গ্রীষ্মকালীন গেমসের আয়োজক)। শহরটি ১৯৫৫ এবং ১৯৭৫ প্যান আমেরিকান গেমস আয়োজন করে, সান্তিয়াগো এবং সাও পাওলো সরে যাওয়ার পর সর্বশেষ। আইসিএফ ফ্লাটওয়াটার রেসিং বিশ্ব চ্যাম্পিয়নশিপ এখানে ১৯৭৪ এবং ১৯৯৪ সালে অনুষ্ঠিত হয়েছিল। লুচা লিব্রে হচ্ছে মেক্সিকোর কুস্তির এক ধরন, এবং এটি সারা দেশে জনপ্রিয় খেলাগুলোর মধ্যে অন্যতম। শহরের প্রধান মাঠগুলি হল এরিনা মেক্সিকো এবং এরিনা কোলিসেও।'
    }
res = nlp(QA_input)
res

{'score': 0.9983528256416321, 'start': 198, 'end': 203, 'answer': ' ১৯৫৫'}